# Convolutional Network 

We're going to build a Convolutional Neural Network from scratch and look at the math behind it.

## What inspired Convolutional Networks?

CNNs are biologically-inspired models inspired by research by D. H. Hubel and T. N. Wiesel. They proposed an explanation for the way in which mammals visually perceive the world around them using a layered architecture of neurons in the brain, and this in turn inspired engineers to attempt to develop similar pattern recognition mechanisms in computer vision.

<img src="assets/inspire.png">

In their hypothesis, within the visual cortex, complex functional responses generated by "complex cells" are constructed from more simplistic responses from "simple cells'. 

For instances, simple cells would respond to oriented edges etc, while complex cells will also respond to oriented edges but with a degree of spatial invariance.

Receptive fields exist for cells, where a cell responds to a summation of inputs from other local cells.

The architecture of deep convolutional neural networks was inspired by the ideas mentioned above 
- local connections 
- layering  
- spatial invariance (shifting the input signal results in an equally shifted output signal. , most of us are able to recognize specific faces under a variety of conditions because we learn abstraction These abstractions are thus invariant to size, contrast, rotation, orientation
 
However, it remains to be seen if these computational mechanisms of convolutional neural networks are similar to the computation mechanisms occurring in the primate visual system

- convolution operation
- shared weights
- pooling/subsampling 


## How it works ?

<img src="assets/cnn_model.jpg">

## Imports

In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
import os
import math
from functools import reduce
import struct
from glob import glob
from random import shuffle

## Step 1 - Prepare a dataset of images

<img src="assets/image.png">

- Every image is a matrix of pixel values. 
- The range of values that can be encoded in each pixel depends upon its bit size. 
- Most commonly, we have 8 bit or 1 Byte-sized pixels. Thus the possible range of values a single pixel can represent is [0, 255]. 
- However, with coloured images, particularly RGB (Red, Green, Blue)-based images, the presence of separate colour channels (3 in the case of RGB images) introduces an additional ‘depth’ field to the data, making the input 3-dimensional. 
- Hence, for a given RGB image of size, say 255×255 (Width x Height) pixels, we’ll have 3 matrices associated with each image, one for each of the colour channels. 
- Thus the image in it’s entirety, constitutes a 3-dimensional structure called the Input Volume (255x255x3).

In [2]:
def load_mnist(path, kind='train'):
    """Load MNIST data from `path`"""
    images_path = glob('./%s/%s*3-ubyte' % (path, kind))[0]
    labels_path = glob('./%s/%s*1-ubyte' % (path, kind))[0]

    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II',
                                 lbpath.read(8))
        labels = np.fromfile(lbpath,
                             dtype=np.uint8)

    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack('>IIII',
                                               imgpath.read(16))
        images = np.fromfile(imgpath,
                             dtype=np.uint8).reshape(len(labels), 784)

    return images, labels

In [3]:
images, labels = load_mnist('./data/mnist')
test_images, test_labels = load_mnist('./data/mnist', 't10k')
np.random.seed(2020)

Now let us look at the result 

# The Neural Network

## Step 2- Define Helper Class

Before creating the Neural Network we will have to define a helper class AverageMeter which will keep track losses and accuracies 

In [4]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

## Step 3 - Define the Convolutional Layer 

![alt text](https://leonardoaraujosantos.gitbooks.io/artificial-inteligence/content/more_images/Convolution_schematic.gif "Logo Title Text 1")

![alt text](http://xrds.acm.org/blog/wp-content/uploads/2016/06/Figure_2.png "Logo Title Text 1")

- A convolution is an orderly procedure where two sources of information are intertwined.

- A kernel (also called a filter) is a smaller-sized matrix in comparison to the input dimensions of the image, that consists of real valued entries.

- Kernels are then convolved with the input volume to obtain so-called ‘activation maps’ (also called feature maps).  
- Activation maps indicate ‘activated’ regions, i.e. regions where features specific to the kernel have been detected in the input. 

- The real values of the kernel matrix change with each learning iteration over the training set, indicating that the network is learning to identify which regions are of significance for extracting features from the data.

- We compute the dot product between the kernel and the input matrix. -The convolved value obtained by summing the resultant terms from the dot product forms a single entry in the activation matrix. 

- The patch selection is then slided (towards the right, or downwards when the boundary of the matrix is reached) by a certain amount called the ‘stride’ value, and the process is repeated till the entire input image has been processed. - The process is carried out for all colour channels.

- instead of connecting each neuron to all possible pixels, we specify a 2 dimensional region called the ‘receptive field[14]’ (say of size 5×5 units) extending to the entire depth of the input (5x5x3 for a 3 colour channel input), within which the encompassed pixels are fully connected to the neural network’s input layer. It’s over these small regions that the network layer cross-sections (each consisting of several neurons (called ‘depth columns’)) operate and produce the activation map. (reduces computational complexity)

![alt text](http://i.imgur.com/g4hRI6Z.png "Logo Title Text 1")
![alt text](http://i.imgur.com/tpQvMps.jpg "Logo Title Text 1")
![alt text](http://i.imgur.com/oyXkhHi.jpg "Logo Title Text 1")
![alt text](http://xrds.acm.org/blog/wp-content/uploads/2016/06/Figure_5.png "Logo Title Text 1")

Great resource on description of  convolution (discrete vs continous)  & the fourier transform

http://timdettmers.com/2015/03/26/convolution-deep-learning/

In [5]:
class Conv2D(object):
    def __init__(self, shape, output_channels, ksize, stride=1, padding=1, weights=None):
        assert len(shape)==4
        assert isinstance(ksize, int) or len(ksize)==2
        self.inp_shape = shape
        self.output_channels = output_channels
        self.padding = padding
        self.input_channels = shape[1]
        self.input_height = shape[2]
        self.input_width = shape[3]
        self.bs = shape[0]
        self.stride = stride
        self.ksize = (ksize,ksize) if isinstance(ksize, int) else ksize
        assert (self.input_height-self.ksize[0]+2*padding)%stride==0
        assert (self.input_width-self.ksize[1]+2*padding)%stride==0
        self.output_height = (self.input_height-self.ksize[0]+2*padding)//stride+1
        self.output_width = (self.input_width-self.ksize[1]+2*padding)//stride+1

        # self.weights = np.random.standard_normal((self.output_channels, self.input_channels,
        #                                           ksize[0], ksize[1]))
        # self.bias = np.random.standard_normal(self.output_channels)
        fan_in = self.input_channels*self.ksize[0]*self.ksize[1]
        self.weights = np.random.standard_normal((self.output_channels, self.input_channels, self.ksize[0], self.ksize[1]))/np.sqrt(fan_in/2)
        self.bias= np.zeros(self.output_channels)

        self.delta = np.zeros((self.bs, self.output_channels, self.output_height, self.output_width))

        self.w_gradient = np.zeros_like(self.weights)
        self.b_gradient = np.zeros_like(self.bias)
        self.weights_m = np.zeros_like(self.weights)
        self.bias_m = np.zeros_like(self.bias)

        self.output_shape = self.delta.shape
        self.i, self.j, self.k = self.get_im2col_slice(x_shape=shape, ksize=self.ksize)

    def forward(self, x, w=None, b=None):
        '''
        convolution forward
        :param x: input, shape is [batch_size, channels, height, width]
        :return:
        '''
        if w is not None:
            self.weights = w
        if b is not None:
            self.bias = b
        weight_cols = self.weights.reshape(self.output_channels, -1)  # 64,27
        self.x_pad = x
        if self.padding!=0:
            self.x_pad = np.pad(x, ((0,0), (0,0),(self.padding, self.padding),
                                    (self.padding, self.padding)), mode="constant")
        img_cols = self.x_pad[:,self.k, self.i, self.j]
        img_cols = img_cols.transpose(1,0,2)
        img_cols = img_cols.reshape(img_cols.shape[0], -1)  # 27,50176
        out = np.dot(weight_cols, img_cols).transpose(1,0)+self.bias    # 50176,64
        out = out.reshape(self.bs, self.output_height, self.output_width, -1).transpose(0,3,1,2)

        cache = (x, self.weights, self.bias, img_cols)
        return out, cache

    def gradient(self, delta, cache):
        x, self.weights, self.bias, img_cols = cache
        stride = self.stride
        padding = self.padding

        delta_reshape = delta.transpose(1,0,2,3)
        delta_reshape = delta_reshape.reshape(delta_reshape.shape[0], -1)
        self.w_gradient = np.dot(delta_reshape, img_cols.T).reshape(self.w_gradient.shape)
        self.b_gradient = np.sum(delta_reshape,axis=1)

        # backward to get the previous layer's delta
        num_filters, _, f_h, f_w = self.weights.shape
        dx_cols = self.weights.reshape(num_filters, -1).T.dot(delta_reshape)
        dx = self.col2im(dx_cols, x.shape, ksize=self.ksize)

        # delta_pad = delta
        # if self.padding!=0:
        #     delta_pad = np.pad(delta, ((0,0), (0,0), (self.padding, self.padding),
        #                        (self.padding, self.padding)), mode="constant")
        # delta_cols = delta_pad[:,self.k, self.i, self.j]    # 4,27,50176
        # delta_cols = delta_cols.transpose(1,0,2)
        # delta_cols = delta_cols.reshape(delta_cols.shape[0], -1) #27,200704
        # fliped_weight = np.flip(np.flip(self.weights, axis=2), axis=3)
        # fliped_weight_cols = fliped_weight.reshape(self.output_channels, -1)  # 64,27
        # prev_delta = np.dot(fliped_weight_cols, delta_cols)    # 64,200704
        # prev_delta = prev_delta.reshape()

        return self.w_gradient, self.b_gradient, dx


    def get_im2col_slice(self, x_shape, ksize):
        k_h, k_w = ksize
        bs, c, i_h, i_w = x_shape
        i0 = np.repeat(np.arange(k_h), k_w)
        i0 = np.tile(i0, c)
        i1 = self.stride*np.repeat(np.arange(self.output_height), self.output_width)
        i= i0.reshape(-1,1)+i1.reshape(1,-1)

        j0 = np.tile(np.arange(k_w), k_h*c)
        j1 = self.stride*np.tile(np.arange(self.output_width), self.output_height)
        j=j0.reshape(-1,1)+j1.reshape(1,-1)

        k = np.repeat(np.arange(c), k_w*k_h).reshape(-1,1)

        return i,j,k

    def col2im(self, cols, x_shape, ksize):

        padding = self.padding
        N, C, H, W = x_shape
        field_height, field_width = ksize
        H_padded, W_padded = H + 2 * padding, W + 2 * padding
        x_padded = np.zeros((N, C, H_padded, W_padded), dtype=cols.dtype)

        i,j,k = self.get_im2col_slice(x_shape, (field_height, field_width))
        cols_reshaped = cols.T.reshape(N, C * field_height * field_width, -1)
        # cols_reshaped = cols_reshaped.transpose(2, 1, 0)
        np.add.at(x_padded, (slice(None), k, i, j), cols_reshaped)
        if padding == 0:
            return x_padded

        return x_padded[:, :, padding:-padding, padding:-padding]

    def backward(self, alpha=1e-4, wd=4e-4, momentum=0.9):
        self.weights *= (1 - wd)
        self.bias *= (1 - wd)
        self.weights_m = momentum*self.weights_m-alpha*self.w_gradient
        self.bias_m = momentum*self.bias_m-alpha*self.b_gradient
        self.weights += self.weights_m
        self.bias += self.bias_m

        self.w_gradient = np.zeros_like(self.w_gradient)
        self.b_gradient = np.zeros_like(self.b_gradient)

## Step 5 - Define the Pooling Layer

![alt text](http://xrds.acm.org/blog/wp-content/uploads/2016/06/Figure_6.png "Logo Title Text 1")

- Pooling reducing the spatial dimensions (Width x Height) of the Input Volume for the next Convolutional Layer. It does not affect the depth dimension of the Volume.  
- The transformation is either performed by taking the maximum value from the values observable in the window (called ‘max pooling’), or by taking the average of the values. Max pooling has been favoured over others due to its better performance characteristics.
- also called downsampling


In [6]:
class MaxPool(object):
    def __init__(self, pool_h, pool_w, stride):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride

    def forward(self, x):
        assert len(x.shape)==4
        (N,C,H,W) = x.shape
        pool_h = self.pool_h
        pool_w = self.pool_w
        stride = self.stride
        H_prime = (H-pool_h)//stride+1
        W_prime = (W-pool_w)//stride+1

        out = np.zeros((N,C,H_prime,W_prime))

        for n in range(N):
            for h in range(H_prime):
                for w in range(W_prime):
                    h1 = h*stride
                    h2 = h1 + pool_h
                    w1 = w*stride
                    w2 = w1 + pool_w
                    window = x[n,:,h1:h2, w1:w2]
                    out[n,:,h,w] = np.max(np.reshape(window, newshape=(C, -1)), axis=1)

        cache = x
        return out, cache

    def gradient(self, dout, cache):
        x = cache
        assert len(x.shape)==4
        (N,C,H,W) = x.shape
        pool_h = self.pool_h
        pool_w = self.pool_w
        stride = self.stride
        H_prime = (H-pool_h)//stride+1
        W_prime = (W-pool_w)//stride+1

        dx = np.zeros_like(x)

        for n in range(N):
            for c in range(C):
                for h in range(H_prime):
                    for w in range(W_prime):
                        h1 = h*stride
                        h2 = h1+pool_h
                        w1 = w*stride
                        w2 = w1+pool_w
                        window1 = x[n,c,h1:h2, w1:w2].reshape(-1)
                        window2 = np.zeros_like(window1)
                        window2[np.argmax(window1)]=1
                        window3 = window2.reshape(pool_h, pool_w)

                        dx[n,c,h1:h2,w1:w2] = window3*dout[n,c,h,w]

        return dx

## Step 6 - Define Dropout, Activation and Batch Norm Layers

Dropout is a technique where randomly selected neurons are ignored during training. They are “dropped-out” randomly. This means that their contribution to the activation of downstream neurons is temporally removed on the forward pass and any weight updates are not applied to the neuron on the backward pass.
![alt text](https://miro.medium.com/max/1026/1*dEi_IkVB7IpkzZ-6H0Vpsg.png)

Batch normalization is a technique for training very deep neural networks that standardizes the inputs to a layer for each mini-batch. We use Batch Normalization for two reasons:
- We can use higher learning rates because batch normalization makes sure that there’s no activation that’s gone really high or really low. And by that, things that previously couldn’t get to train, it will start to train.

- It reduces overfitting because it has a slight regularization effects. Similar to dropout, it adds some noise to each hidden layer’s activations. Therefore, if we use batch normalization, we will use less dropout, which is a good thing because we are not going to lose a lot of information. However, we should not depend only on batch normalization for regularization; we should better use it together with dropout.

For the Activation Layer we will be using ReLu due to its easier computability

In [7]:
# Dropout Layer to prevent Overfitting
class Dropout(object):
    def __init__(self, keep_rate):
        self.keep_rate = keep_rate

    def forward(self, x, training=True):
        if training:
            mask = np.random.binomial(1, self.keep_rate, size=x.shape)/self.keep_rate
            mask_x = x*mask

            cache = mask
            return mask_x, cache
        else:
            return x

    def gradient(self, dout, cache):
        mask = cache
        dx = dout*mask
        return dx

class ReLU(object):
    def forward(self, x):
        out = np.maximum(0, x)
        cache = x
        return out, cache

    def gradient(self, dout, cache):
        x = cache
        dx = np.array(dout)
        dx[x<=0] = 0
        return dx    
    
# Normalization for Convolutional Layers
class BatchNorm2D(object):
    def __init__(self, num_features, affine=True, moving_decay=0.9, epsilon=1e-8):
        '''
        Apply to convolution layer
        :param num_features:
        :param affine:
        :param moving_decay:
        :param epsilon:
        '''
        self.num_features = num_features
        self.affine = affine
        self.epsilon = epsilon
        self.moving_decay = moving_decay
        if self.affine:
            self.weight = np.ones((num_features,))
            self.bias = np.zeros((num_features,))
        self.moving_mean = np.zeros((num_features,))
        self.moving_var = np.ones((num_features,))
        self.mean = np.zeros((num_features,))
        self.var = np.ones((num_features,))
        self.weights_m = np.zeros_like(self.weight)
        self.bias_m = np.zeros_like(self.bias)

    def forward(self, x, training=True):
        assert len(x.shape)==4
        B = x.shape[0]
        mean = np.mean(x, axis=(0,2,3))
        var = np.var(x, axis=(0,2,3))
        self.mean = mean
        self.var = var
        if training:
            if np.mean(self.moving_mean)==0.0 and np.mean(self.moving_var)==1.0:
                self.moving_mean = mean
                self.moving_var = var
            else:
                self.moving_mean = self.moving_mean*self.moving_decay+(1-self.moving_decay)*mean
                self.moving_var = self.moving_var*self.moving_decay+(1-self.moving_decay)*var
            normx = (x - mean[np.newaxis,:,np.newaxis,np.newaxis])/np.sqrt(var[np.newaxis,:,np.newaxis,np.newaxis]+self.epsilon)
        else:
            normx = (x - self.moving_mean[np.newaxis,:,np.newaxis,np.newaxis])/np.sqrt(self.moving_var[np.newaxis,:,np.newaxis,np.newaxis]+self.epsilon)

        cache = (x, normx)

        return normx*self.weight[np.newaxis,:,np.newaxis,np.newaxis]+self.bias[np.newaxis,:,np.newaxis,np.newaxis], cache

    def gradient(self, dout, cache):
        x, normx = cache
        B = x.shape[0]*x.shape[2]*x.shape[3]
        mean = self.mean
        var = self.var
        x_mu = x-mean[np.newaxis,:,np.newaxis,np.newaxis]

        d_normx = dout*self.weight[np.newaxis,:,np.newaxis,np.newaxis]
        d_var = np.sum(d_normx*x_mu, axis=(0,2,3))*-.5*(var+self.epsilon)**(-3/2)
        d_mu = np.sum(d_normx*-1/np.sqrt(var+self.epsilon)[np.newaxis,:,np.newaxis,np.newaxis], axis=(0,2,3))+d_var*np.sum(-2*x_mu,axis=(0,2,3))/B
        d_x = d_normx/np.sqrt(var+self.epsilon)[np.newaxis,:,np.newaxis,np.newaxis]+\
              d_var[np.newaxis,:,np.newaxis,np.newaxis]*2*x_mu/B+\
              d_mu[np.newaxis,:,np.newaxis,np.newaxis]/B
        self.d_weight = np.sum(dout*normx, axis=(0,2,3))
        self.d_bias = np.sum(dout, axis=(0,2,3))

        return self.d_weight, self.d_bias, d_x

    def backward(self, alpha=1e-4, momentum=0.9):
        self.weights_m = momentum * self.weights_m - alpha * self.d_weight
        self.bias_m = momentum * self.bias_m - alpha * self.d_bias
        self.weight += self.weights_m
        self.bias += self.bias_m

        self.d_weight = np.zeros_like(self.d_weight)
        self.d_bias = np.zeros_like(self.d_bias)

# Normalization for Linear layer
class BatchNorm1D(object):
    def __init__(self, num_features, affine=True, moving_decay=0.99, epsilon=1e-8):
        '''
        Apply to the linear layer
        :param num_features:
        :param affine:
        :param moving_decay:
        :param epsilon:
        '''
        self.num_features = num_features
        self.affine = affine
        self.moving_decay = moving_decay
        self.epsilon = epsilon
        if self.affine:
            self.weight = np.ones((num_features,))
            self.bias = np.zeros((num_features,))
        self.moving_mean = np.zeros((num_features,))
        self.moving_var = np.ones((num_features,))
        self.mean = np.zeros((num_features,))
        self.var = np.ones((num_features,))

    def forward(self, x, training=True):
        assert len(x.shape)==2
        B = x.shape[0]
        mean = np.mean(x, axis=0)
        var = np.var(x, axis=0)
        self.mean = mean
        self.var = var
        if training:
            if np.mean(self.moving_mean) == 0.0 and np.mean(self.moving_var) == 1.0:
                self.moving_mean = mean
                self.moving_var = var
            else:
                self.moving_mean = self.moving_mean * self.moving_decay + (1 - self.moving_decay) * mean
                self.moving_var = self.moving_var * self.moving_decay + (1 - self.moving_decay) * var
            normx = (x - mean[np.newaxis, :]) / np.sqrt(var[np.newaxis, :] + self.epsilon)
        else:
            normx = (x - self.moving_mean[np.newaxis, :]) / np.sqrt(self.moving_var[np.newaxis, :] + self.epsilon)

        cache = (x, normx)

        return normx * self.weight[np.newaxis, :] + self.bias[np.newaxis, :], cache

    def gradient(self, dout, cache):
        x, normx = cache
        B = x.shape[0]
        mean = self.mean
        var = self.var
        x_mu = x - mean[np.newaxis, :]

        d_normx = dout * self.weight[np.newaxis, :]
        d_var = np.sum(d_normx * x_mu, axis=0) * -.5 * (var + self.epsilon) ** (-3 / 2)
        d_mu = np.sum(d_normx * -1 / np.sqrt(var + self.epsilon)[np.newaxis, :],
                      axis=0) + d_var * np.sum(-2 * x_mu, axis=0) / B
        d_x = d_normx / np.sqrt(var + self.epsilon)[np.newaxis, :] + \
              d_var[np.newaxis, :] * 2 * x_mu / B + \
              d_mu[np.newaxis, :] / B
        self.d_weight = np.sum(dout * normx, axis=0)
        self.d_bias = np.sum(dout, axis=0)

        return self.d_weight, self.d_bias, d_x

## Step 6 - Define the Final Layers 

![alt text](https://miro.medium.com/max/1400/1*IWUxuBpqn2VuV-7Ubr01ng.png)

The output from the convolutional layers is passed through this layer which flattens it into a 1-dimensional array for inputting it to the next layer. We flatten the output of the convolutional layers to create a single long feature vector. And it is connected to the final classification model, which is called a fully-connected layer. In other words, we put all the pixel data in one line and make connections with the final layer which will be linear. 

In [8]:
#Fully Connected Layer
class Linear(object):
    def __init__(self, in_channels, out_channels):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.W = np.random.standard_normal((in_channels, out_channels))/np.sqrt(in_channels/2)
        self.b = np.zeros((out_channels,))
        self.gradient_W = np.zeros_like(self.W)
        self.gradient_b = np.zeros_like(self.b)
        self.weights_m = np.zeros_like(self.W)
        self.bias_m = np.zeros_like(self.b)

    def forward(self, x, w=None, b=None):
        if w is not None:
            self.W = w
        if b is not None:
            self.b = b
        assert len(x.shape)==2
        out = np.dot(x, self.W)+self.b
        return out, x

    def gradient(self, dout, cache):
        x = cache
        self.gradient_W = np.dot(x.T, dout)
        self.gradient_b = np.sum(dout, 0)

        dx = np.dot(dout, self.W.T)

        return self.gradient_W, self.gradient_b, dx

    def backward(self, alpha=1e-4, wd=4e-4, momentum = 0.9):
        self.W*=(1-wd)
        self.b*=(1-wd)

        self.weights_m = momentum * self.weights_m - alpha * self.gradient_W
        self.bias_m = momentum * self.bias_m - alpha * self.gradient_b

        self.W += self.weights_m
        self.b += self.bias_m

        self.gradient_W = np.zeros_like(self.gradient_W)
        self.gradient_b = np.zeros_like(self.gradient_b)

    
# Final Output Layer
class Softmax_and_Loss(object):
    """
    Compute the softmax and loss together
    """
    def forward_and_backward(self, x, y):
        """
        forward and backward
        :param x: final layer output, before softmax layer, [N,C]
        :param y: ground truth, [N,]
        :return:
        """
        probs = np.exp(x - np.max(x, axis=1, keepdims=True))
        probs /= np.sum(probs, axis=1, keepdims=True)
        N = x.shape[0]
        loss = -np.sum(np.log(probs[np.arange(N),y]))/N
        dx = probs.copy()
        dx[np.arange(N), y]-=1.0

        dx/=N   # for the loss is divided by N
        return loss, dx

## Step 8 - Define the Neural Network

Now we can define a Neural Network class that can be used to combine all the layers defined previously. The class will also contain the fit function that will be used to train the network using the calculated gradients by each of the layers

In [9]:
batch_size = 128
init_lr = 0.1
# Define network
conv1 = Conv2D(shape=[batch_size, 1, 32, 32], output_channels=6, ksize=5, stride=1, padding=0)    #28*28
bn1 = BatchNorm2D(6)
relu1 = ReLU()
pool1 = MaxPool(2,2,2)  #14*14
conv2 = Conv2D(shape=[batch_size, 6, 14, 14], output_channels=16, ksize=5, stride=1, padding=0)    #10*10
bn2 = BatchNorm2D(16)
relu2 = ReLU()
pool2 = MaxPool(2,2,2)  # 5*5
conv3 = Conv2D(shape=[batch_size, 16, 5, 5], output_channels=120, ksize=5, stride=1, padding=0)    #1*1
bn3 = BatchNorm2D(120)
relu3 = ReLU()
fc1 = Linear(1*1*120, 84)
relu4 = ReLU()
dp = Dropout(0.9)
fc2 = Linear(84, 10)
sf = Softmax_and_Loss()

## Final Step - Define Variables and Train the Network

In [10]:
step=0
train_loss = AverageMeter()
train_acc = AverageMeter()
val_loss = AverageMeter()
val_acc = AverageMeter()
train_loss_all=[]
val_loss_all=[]
train_acc_all=[]
val_acc_all=[]

In [11]:
for epoch in range(1):
    index = [i for i in range(images.shape[0])]
    shuffle(index)
    images = images[index]
    labels = labels[index]
    # train
    for i in range(images.shape[0] // batch_size):
        step+=1
        # learning rate decay
        lr = init_lr*0.9**(step//30)
        # forward
        img = images[i * batch_size:(i + 1) * batch_size].reshape([batch_size, 28, 28, 1]).transpose(0,3,1,2)
        img = np.pad(img, [[0,0],[0,0],[2,2],[2,2]], mode='constant')/255.0
        img = (img-0.1307)/0.3081
        label = labels[i * batch_size:(i + 1) * batch_size]
        out, conv1_cache = conv1.forward(img)
        out, bn1_cache = bn1.forward(out)
        out, relu1_cache = relu1.forward(out)
        out, pool1_cache = pool1.forward(out)

        out, conv2_cache = conv2.forward(out)
        out, bn2_cache = bn2.forward(out)
        out, relu2_cache = relu2.forward(out)
        out, pool2_cache = pool2.forward(out)

        out, conv3_cache = conv3.forward(out)
        out, bn3_cache = bn3.forward(out)
        out, relu3_cache = relu3.forward(out)

        conv_out = out

        out = conv_out.reshape(batch_size, -1)
        out, fc1_cache = fc1.forward(out)
        out, relu4_cache = relu4.forward(out)
        out, dp_cache = dp.forward(out)

        out, fc2_cache = fc2.forward(out)
        loss, dx =  sf.forward_and_backward(out, np.array(label))

        # calculate gradient
        _, _, dx = fc2.gradient(dx, fc2_cache)

        dx = dp.gradient(dx, dp_cache)
        dx = relu4.gradient(dx, relu4_cache)
        _,_,dx = fc1.gradient(dx, fc1_cache)

        dx = dx.reshape(conv_out.shape)

        dx = relu3.gradient(dx, relu3_cache)
        _,_,dx = bn3.gradient(dx, bn3_cache)
        _,_,dx = conv3.gradient(dx, conv3_cache)


        dx = pool2.gradient(dx, pool2_cache)
        dx = relu2.gradient(dx, relu2_cache)
        _, _, dx = bn2.gradient(dx, bn2_cache)
        _, _ ,dx = conv2.gradient(dx, conv2_cache)

        dx = pool1.gradient(dx, pool1_cache)
        dx = relu1.gradient(dx, relu1_cache)
        _, _, dx = bn1.gradient(dx, bn1_cache)
        _,_, dx = conv1.gradient(dx, conv1_cache)

        # backward
        conv1.backward(lr)
        bn1.backward(lr)
        conv2.backward(lr)
        bn2.backward(lr)
        conv3.backward(lr)
        bn3.backward(lr)
        fc1.backward(lr)
        fc2.backward(lr)
        
        
        pred = np.argmax(out, axis=1)
        correct = pred.__eq__(label).sum()
        train_acc.update(correct/label.size*100)
        train_loss.update(loss)


        print(epoch,i,lr, train_loss.avg,train_acc.avg)

        if i%10==0:
            for k in range(test_images.shape[0] // batch_size):
                batch_acc = 0
                img = test_images[k * batch_size:(k + 1) * batch_size].reshape([batch_size, 28, 28, 1]).transpose(0, 3, 1, 2)
                img = np.pad(img, [[0, 0], [0, 0], [2, 2], [2, 2]], mode='constant')/255.0
                img = (img-0.1307)/0.3081
                label = test_labels[k * batch_size:(k + 1) * batch_size]

                out, conv1_cache = conv1.forward(img)
                out, bn1_cache = bn1.forward(out, False)
                out, relu1_cache = relu1.forward(out)
                out, pool1_cache = pool1.forward(out)

                out, conv2_cache = conv2.forward(out)
                out, bn2_cache = bn2.forward(out, False)
                out, relu2_cache = relu2.forward(out)
                out, pool2_cache = pool2.forward(out)
                out, conv3_cache = conv3.forward(out)
                out, bn3_cache = bn3.forward(out, False)
                out, relu3_cache = relu3.forward(out)

                conv_out = out

                out = conv_out.reshape(batch_size, -1)
                out, fc1_cache = fc1.forward(out)
                out, relu4_cache = relu4.forward(out)
                out = dp.forward(out, False)

                out, fc2_cache = fc2.forward(out)
                loss, dx =  sf.forward_and_backward(out, np.array(label))
                
                pred = np.argmax(out, axis=1)
                correct = pred.__eq__(label).sum()
                val_acc.update(correct/label.size*100)
                val_loss.update(loss)
            print("val loss:", val_loss.avg, "val acc:", val_acc.avg)
            
            train_acc_all.append(train_acc.avg)
            train_loss_all.append(train_loss.avg)
            val_acc_all.append(val_acc.avg)
            val_loss_all.append(val_loss.avg)
            
            val_loss.reset()
            val_acc.reset()
            train_acc.reset()
            train_loss.reset()

0 0 0.1 3.1540872135852966 6.25
val loss: 2.3348269844157747 val acc: 18.67988782051282
0 1 0.1 2.4771103255357714 11.71875
0 2 0.1 2.1328266004634946 28.125
0 3 0.1 2.017153303372607 34.375
0 4 0.1 1.853635223016993 41.015625
0 5 0.1 1.7381624984098238 46.09375
0 6 0.1 1.6132165778212546 50.911458333333336
0 7 0.1 1.511949545977372 54.129464285714285
0 8 0.1 1.4139728087495063 57.03125
0 9 0.1 1.3165090180298387 60.416666666666664
0 10 0.1 1.2413694061714469 62.5
val loss: 1.363293542458472 val acc: 60.31650641025641
0 11 0.1 0.436363269029711 92.1875
0 12 0.1 0.47762266210718907 87.5
0 13 0.1 0.49588191699575823 85.9375
0 14 0.1 0.4701003617991034 86.1328125
0 15 0.1 0.47244973112771305 85.625
0 16 0.1 0.4580994433464472 86.06770833333333
0 17 0.1 0.46187849999591685 85.82589285714286
0 18 0.1 0.4377866734457216 86.42578125
0 19 0.1 0.4506142937886967 86.19791666666667
0 20 0.1 0.45158502960469155 86.09375
val loss: 0.9891267621648621 val acc: 74.69951923076923
0 21 0.1 0.43229119488

0 152 0.05904900000000001 0.16011923511877202 94.53125
0 153 0.05904900000000001 0.184883916504562 94.27083333333333
0 154 0.05904900000000001 0.1903653934338602 93.9453125
0 155 0.05904900000000001 0.17301309620773936 95.0
0 156 0.05904900000000001 0.16982731672499551 94.79166666666667
0 157 0.05904900000000001 0.15888442071165385 95.08928571428571
0 158 0.05904900000000001 0.14842805940596318 95.41015625
0 159 0.05904900000000001 0.14125480509740057 95.74652777777777
0 160 0.05904900000000001 0.13384257328442128 95.9375
val loss: 0.09685911336237951 val acc: 96.77483974358974
0 161 0.05904900000000001 0.10824250160076984 96.875
0 162 0.05904900000000001 0.11791575600744401 96.484375
0 163 0.05904900000000001 0.10997075566548054 96.875
0 164 0.05904900000000001 0.09907966163389045 96.875
0 165 0.05904900000000001 0.0951772351769137 97.1875
0 166 0.05904900000000001 0.10484955824764804 96.74479166666667
0 167 0.05904900000000001 0.09816771522594246 96.875
0 168 0.05904900000000001 0.09

0 282 0.03874204890000001 0.09804179675004193 97.65625
0 283 0.03874204890000001 0.08807347990164026 97.65625
0 284 0.03874204890000001 0.0819673631424076 97.65625
0 285 0.03874204890000001 0.09527203238953341 97.34375
0 286 0.03874204890000001 0.08664452755274525 97.52604166666667
0 287 0.03874204890000001 0.08712574656820189 97.54464285714286
0 288 0.03874204890000001 0.08934160739017233 97.65625
0 289 0.03874204890000001 0.08820022528793255 97.65625
0 290 0.03874204890000001 0.08768864179708936 97.578125
val loss: 0.06764815329395468 val acc: 97.71634615384616
0 291 0.03874204890000001 0.0698288674775282 99.21875
0 292 0.03874204890000001 0.04470730426907987 99.21875
0 293 0.03874204890000001 0.05990864109661844 98.69791666666667
0 294 0.03874204890000001 0.05549427170189013 98.828125
0 295 0.03874204890000001 0.05440410766323509 98.59375
0 296 0.03874204890000001 0.07109807527269493 97.91666666666667
0 297 0.03874204890000001 0.07874862689316767 97.54464285714286
0 298 0.0387420489

val loss: 0.058426848213681576 val acc: 98.15705128205128
0 411 0.02541865828329001 0.10642414450768986 98.4375
0 412 0.02541865828329001 0.0735460838152383 98.828125
0 413 0.02541865828329001 0.06687960489632887 98.95833333333333
0 414 0.02541865828329001 0.06194828577964619 99.0234375
0 415 0.02541865828329001 0.06857503837310912 98.59375
0 416 0.02541865828329001 0.0699946439579595 98.30729166666667
0 417 0.02541865828329001 0.07150057352904908 98.32589285714286
0 418 0.02541865828329001 0.07074863663548478 98.14453125
0 419 0.02287679245496101 0.06915345506104577 98.09027777777777
0 420 0.02287679245496101 0.0712404887787128 98.125
val loss: 0.05548056907914125 val acc: 98.23717948717949
0 421 0.02287679245496101 0.031108229971646487 100.0
0 422 0.02287679245496101 0.029117237111116103 99.609375
0 423 0.02287679245496101 0.043216096914059025 98.69791666666667
0 424 0.02287679245496101 0.03907499082036929 98.828125
0 425 0.02287679245496101 0.045460641881898384 98.59375
0 426 0.0228

The CNN shows a training accuracy of 98.66% and validation accuracy of 98.15% after just one epoch of 